In [1]:
from sympy import symbols, integrate, Matrix, solve, sin, pi
import numpy as np
import matplotlib.pyplot as plt

# Define constants
E = 200e9  # Pa
nu = 0.3
h = 0.2  # m
q0 = 5e3  # N/m^2
a = 8  # m
b = 8  # m
D = E * h**3 / (12 * (1 - nu**2))  # Flexural rigidity ≈ 1.4652e8 N·m

# Define symbolic variables
x, y = symbols('x y')
c11, c12, c21, c22 = symbols('c11 c12 c21 c22')

# Basis functions for Ritz method
phi = [(sin(i * pi * x / a) * sin(j * pi * y / b)) for i in [1, 2] for j in [1, 2]]


In [2]:
# Compute K_ij and F_i for Ritz method
def compute_system(n):
    K = Matrix.zeros(n, n)
    F = Matrix.zeros(n, 1)
    for i in range(n):
        for j in range(n):
            w_xx = phi[i].diff(x, 2)
            w_yy = phi[i].diff(y, 2)
            v_xx = phi[j].diff(x, 2)
            v_yy = phi[j].diff(y, 2)
            w_xy = phi[i].diff(x, y)
            v_xy = phi[j].diff(x, y)
            integrand = D * (w_xx * v_xx + w_yy * v_yy + nu * (w_xx * v_yy + w_yy * v_xx) + 2 * (1 - nu) * w_xy * v_xy)
            K[i, j] = integrate(integrand, (x, 0, a), (y, 0, b))
        F[i] = integrate(q0 * phi[i], (x, 0, a), (y, 0, b))
    return K, F

In [5]:
# Compute Ritz approximations for n=1 and n=4
solutions = {}
for n in [1, 4]:
    K, F = compute_system(n)
    c = Matrix(symbols(f'c1:{n+1}'))
    solution = solve(K * c - F, c)
    w_approx = sum(solution.get(symbols(f'c{i+1}'), 0) * phi[i] for i in range(n))
    solutions[n] = (K, F, solution, w_approx)

In [6]:
# Analytical solution (Navier’s method)
m, n = symbols('m n', integer=True)
w_mn = (16 * q0 / (pi**2 * m * n)) / (D * pi**4 * ((m**2 / a**2) + (n**2 / b**2))**2) * sin(m * pi * x / a) * sin(n * pi * y / b)
# Sum over odd m, n up to 5
w_analytical = 0
m_max, n_max = 5, 5
for i in range(1, m_max + 1, 2):
    for j in range(1, n_max + 1, 2):
        w_analytical += w_mn.subs({m: i, n: j})

TypeError: loop of ufunc does not support argument 0 of type Zero which has no callable sin method